In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETsARD

In [2]:
loader = PETsARD.Loader(
    filepath='../[sunset]/data/[Adt Income] adult.csv',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
print(loader.data.head(1))


   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [3]:
splitter = PETsARD.Splitter(
    data=loader.data,
    num_samples=1,  # 30,
    train_split_ratio=0.8
)
print(splitter.data[1]['train'].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'].head(1))
print(splitter.data[1]['validation'].head(1))


39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   27   Private  205145   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
preproc = PETsARD.Preprocessor(
    data=splitter.data[1]['train'],
    missing=True,
    missing_method='drop',
    missing_columns=None,
    outlier=True,
    outlier_method='IQR',
    outlier_columns={'ignore': ['hours-per-week']},
    encoding=True,
    encoding_method='Label',
    encoding_columns=None,
    scaling=True,
    scaling_method='Standard',
    scaling_columns={'focus': 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)


Preprocessor - Outlierist (IQR): Dropped  3028 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   224 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1742 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1068 rows on fnlwgt         . Kept [-63123.0, 417765.0] only.
Preprocessor - Outlierist (IQR): Dropped   235 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6089 in 36205 rows.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Lab

In [5]:
synthesizer = PETsARD.Synthesizer(
    data=preproc.data,
    synthesizing_method='sdv-singletable-gaussiancopula'
)
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))


Synthesizer (SDV - SingleTable): Metafile loading time: 0.037 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 14.8843 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 30116 rows (same as raw) in 1.4872 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   49          2 -1.503732          8               10               0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           4             1     2       1             0             0   

   hours-per-week  native-country  income  
0              38              29       0  


In [6]:
# File D:\Dropbox\89_其他應用\GitHub\PETs-Experiment\PETsARD\Preprocessor\Missingist.py:27, in Missingist.inverse_transform(self, col_data)
#      25 _col_data = col_data.copy()
#      26 print(_col_data)
# ---> 27 _col_data[_na_mask] = np.nan
#      28 return _col_data

# ValueError: cannot convert float NaN to integer

postproc = PETsARD.Postprocessor(
    data=synthesizer.data_syn,
    missingist = getattr(preproc ,'missingist' ,None),
    encoder=getattr(preproc, 'encoder', None),
    scaler=getattr(preproc, 'scaler', None)
)
print(postproc.data.head(1))


Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Missingist (<PETsARD.Preprocessor.Missingist_Drop.Missingist_Drop object at 0x00000128A424FE80>): Decoding capital-gain.
Postprocessor - Missingist (<PETsARD.Preprocessor.Missingist_Drop.Missingist_Drop object at 0x00000128A42CD450>): Decoding age.
Postprocessor - Missingist (<PETsARD.Preprocessor.Missingist_Drop.Missingist_Drop object at 0x00000128A42CD300>): Decoding education.


In [7]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-singlingout-univariate', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                                      }, anonymeter_n_attacks=2000
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0816 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.0013568577126237004,
 'Attack_Rate': 0.0009585236406264672,
 'Attack_Rate_err': 0.0009585236406264671,
 'Baseline_Rate': 0.0009585236406264672,
 'Baseline_Rate_err': 0.0009585236406264671,
 'Control_Rate': 0.0009585236406264672,
 'Control_Rate_err': 0.0009585236406264671}

In [8]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-linkability', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                           }, anonymeter_n_attacks=2000, anonymeter_n_neighbors=10, anonymeter_n_jobs=-1, anonymeter_aux_cols=[['age', 'fnlwgt', 'race', 'gender', 'native-country'], ['workclass', 'education', 'capital-gain', 'capital-loss', 'hours-per-week']
                                                                                                                                                                               ]
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Linkability): Now is Linkability Evaluator
Evaluator (Anonymeter - Linkability): aux_cols are [age, fnlwgt, race, gender, native-country]
                                      and [workclass, education, capital-gain, capital-loss, hours-per-week].
Evaluator (Anonymeter - Linkability): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Linkability.
Evaluator (Anonymeter): Evaluating Linkability spent 13.7814 sec.


{'Risk': 0.0015113953698751466,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.007640001127268101,
 'Attack_Rate': 0.010939353167813939,
 'Attack_Rate_err': 0.004456585291261251,
 'Baseline_Rate': 0.001956606593345214,
 'Baseline_Rate_err': 0.00168232767569318,
 'Control_Rate': 0.00944222873873582,
 'Control_Rate_err': 0.004128460999050264}

In [9]:
eval = PETsARD.Evaluator(evaluating_method='anonymeter-inference', data={'ori': splitter.data[1]['train'], 'syn': postproc.data, 'control': splitter.data[1]['validation']
                                                                         }, anonymeter_n_attacks=2000, anonymeter_n_jobs=-1, anonymeter_secret='age'
                         )
eval.eval()
eval.Evaluator.evaluation


Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating Inference spent 0.8025 sec.


{'Risk': 0.026148306735715394,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.06623361606884481,
 'Attack_Rate': 0.11297349278000537,
 'Attack_Rate_err': 0.027482077623402282,
 'Baseline_Rate': 0.0593852071649292,
 'Baseline_Rate_err': 0.020359586481777707,
 'Control_Rate': 0.08915647695108263,
 'Control_Rate_err': 0.024683334547474907}